# Basic CSGO Analysis
##### *Last Updated: January 17, 2022*
The csgo package was developed with easy analysis in mind. To that end, the data parsed goes directly into Pandas DataFrames, as shown in the first example notebook, [Parsing a CSGO demofile](https://github.com/pnxenopoulos/csgo/blob/master/examples/00_Parsing_a_CSGO_Demofile.ipynb). To efficiently calculate aggregate statistics from these Pandas Dataframes, the package contains `calc_stats()`, which filters, groups, and aggregates data based on user input. Furthermore, the package contains thirteen functions derived from `calc_stats()` to calculate standard CSGO aggregate statistics. 

To start, we reference the [demofile](https://www.hltv.org/matches/2337844/astralis-vs-liquid-blast-pro-series-global-final-2019) for a match between Astralis and Team Liquid, where we look at the second map of the series, `nuke`.

In [1]:
import operator
from typing import Dict, List, Tuple, Union

import pandas as pd

from csgo.parser import DemoParser

# Create the parser object.
demo_parser = DemoParser(
    demofile="astralis-vs-liquid-m2-nuke.dem",
    demo_id="AST-TL-BLAST2019",
)

# Parse the demofile, output results to a dictionary of dataframes, and clean the data.
data_df = demo_parser.parse(return_type="df")
data_df = demo_parser.clean_rounds(return_type="df")

09:09:22 [INFO] Initialized CSGODemoParser with demofile D:\MyProfile\Documents\CSGO_Demofiles\astralis-vs-liquid-m2-nuke.dem
09:09:22 [INFO] Setting demo id to AST-TL-BLAST2019
09:09:22 [INFO] Setting parse rate to 128
09:09:22 [INFO] Setting trade time to 5
09:09:22 [INFO] Setting buy style to hltv
09:09:22 [INFO] Rollup damages set to False
09:09:22 [INFO] Parse frames set to True
09:09:22 [INFO] Go version>=1.14.0
09:09:22 [INFO] Running Golang parser from C:\Users\aagrawal-22\Anaconda3\lib\site-packages\csgo-1.0-py3.9.egg\csgo\parser\
09:09:22 [INFO] Looking for file at D:\MyProfile\Documents\CSGO_Demofiles\astralis-vs-liquid-m2-nuke.dem
09:09:29 [INFO] Wrote demo parse output to AST-TL-BLAST2019.json
09:09:30 [INFO] JSON data loaded, available in the `json` attribute to parser
09:09:30 [INFO] JSON output found
09:09:30 [INFO] Returned dataframe output
09:09:30 [INFO] Returned dataframe output
09:09:34 [INFO] Returned dataframe output
09:09:34 [INFO] Returned cleaned dataframe out

## `calc_stats()` 
`calc_stats()` can be used to calculate aggregate statistics from any of the Pandas DataFrames containing event data. It also allows the user to pass column filters. For example, we can use the function to calculate each player's headshot kills in the first half.

In [2]:
# Helper functions for calc_stats()
def extract_num_filters(
    filters: Dict[str, Union[List[bool], List[str]]], key: str
) -> Tuple[List[str], List[float]]:
    sign_list = []
    val_list = []
    for index in filters[key]:
        if not isinstance(index, str):
            raise ValueError(
                f'Filter(s) for column "{key}" must be of type ' f"string."
            )
        i = 0
        sign = ""
        while i < len(index) and not index[i].isdecimal():
            sign += index[i]
            end_index = i
            i += 1
        if sign not in ("==", "!=", "<=", ">=", "<", ">"):
            raise Exception(
                f'Invalid logical operator in filters for "{key}"' f" column."
            )
        sign_list.append(sign)
        try:
            val_list.append(float(index[end_index + 1:]))
        except ValueError as ve:
            raise Exception(
                f'Invalid numerical value in filters for "{key}" ' f"column."
            ) from ve
    return sign_list, val_list


def check_filters(df: pd.DataFrame, filters: Dict[str, Union[List[bool], List[str]]]):
    for key in filters:
        if df.dtypes[key] == "bool":
            for index in filters[key]:
                if not isinstance(index, bool):
                    raise ValueError(
                        f'Filter(s) for column "{key}" must be ' f"of type boolean"
                    )
        elif df.dtypes[key] == "O":
            for index in filters[key]:
                if not isinstance(index, str):
                    raise ValueError(
                        f'Filter(s) for column "{key}" must be ' f"of type string"
                    )
        else:
            extract_num_filters(filters, key)


def num_filter_df(df: pd.DataFrame, col: str, sign: str, val: float) -> pd.DataFrame:
    ops = {
        "==": operator.eq(df[col], val),
        "!=": operator.ne(df[col], val),
        "<=": operator.le(df[col], val),
        ">=": operator.ge(df[col], val),
        "<": operator.lt(df[col], val),
        ">": operator.gt(df[col], val),
    }
    filtered_df = df.loc[ops[sign]]
    return filtered_df


def filter_df(
    df: pd.DataFrame, filters: Dict[str, Union[List[bool], List[str]]]
) -> pd.DataFrame:
    df_copy = df.copy()
    check_filters(df_copy, filters)
    for key in filters:
        if df_copy.dtypes[key] == "bool" or df_copy.dtypes[key] == "O":
            df_copy = df_copy.loc[df_copy[key].isin(filters[key])]
        else:
            i = 0
            for sign in extract_num_filters(filters, key)[0]:
                val = extract_num_filters(filters, key)[1][i]
                df_copy = num_filter_df(
                    df_copy, key, extract_num_filters(filters, key)[0][i], val
                )
                i += 1
    return df_copy

In [3]:
def calc_stats(
    df: pd.DataFrame,
    filters: Dict[str, Union[List[bool], List[str]]],
    col_to_groupby: List[str],
    col_to_agg: List[str],
    agg: List[List[str]],
    col_names: List[str],
) -> pd.DataFrame:
    df_copy = filter_df(df, filters)
    agg_dict = dict(zip(col_to_agg, agg))
    if col_to_agg:
        df_copy = df_copy.groupby(col_to_groupby).agg(agg_dict).reset_index()
    df_copy.columns = col_names
    return df_copy

Below, the data is set to the `kills` DataFrame, the data is filtered to where the value of the column `isHeadshot` is True and the value of the column `roundNum` is less than 16, the data is grouped by `attackerName`, the column `attackerName` is aggregated, the aggregation function `size()` is used, and the columns are renamed to `Player` and `1st Half HS`.

In [4]:
calc_stats(data_df["kills"], {"isHeadshot": [True], "roundNum": ["<16"]},
           ["attackerName"], ["attackerName"], [["size"]],
           ["Player", "1st Half HS"])

,Player,1st Half HS
0,EliGE,8
1,Magisk,3
2,NAF,1
3,Stewie2K,6
4,Twistzz,5
5,Xyp9x,2
6,device,7
7,dupreeh,5
8,gla1ve,2
9,nitr0,12


As mentioned earlier, the package contains thirteen functions derived from `calc_stats()` to efficiently calculate popular CSGO aggregate statistics. Unlike `calc_stats()`, the columns to group and aggregate the data by, the aggregation functions, and the column names do not need to be passed to these functionns; only the data and column filters need to be passed.

# `accuracy()`
`accuracy()` takes in damage data, weapon fire data, a boolean specifying whether to calculate statistics for each player or for each team, and filters for each group of data, and returns a DataFrame with weapon fires, strafe percentage, accuracy percentage, and headshot accuracy percentage.

In [5]:
def accuracy(
    damage_data: pd.DataFrame,
    weapon_fire_data: pd.DataFrame,
    team: bool = False,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    weapon_fire_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    stats = ["playerName", "attackerName", "Player"]
    if team:
        stats = ["playerTeam", "attackerTeam", "Team"]
    weapon_fires = calc_stats(
        weapon_fire_data,
        weapon_fire_filters,
        [stats[0]],
        [stats[0]],
        [["size"]],
        [stats[2], "Weapon Fires"],
    )
    strafe_fires = calc_stats(
        weapon_fire_data.loc[weapon_fire_data["playerStrafe"] == True],
        weapon_fire_filters,
        [stats[0]],
        [stats[0]],
        [["size"]],
        [stats[2], "Strafe Fires"],
    )
    hits = calc_stats(
        damage_data.loc[damage_data["attackerTeam"]
                        != damage_data["victimTeam"]],
        damage_filters,
        [stats[1]],
        [stats[1]],
        [["size"]],
        [stats[2], "Hits"],
    )
    headshots = calc_stats(
        damage_data.loc[
            (damage_data["attackerTeam"] != damage_data["victimTeam"])
            & (damage_data["hitGroup"] == "Head")
        ],
        damage_filters,
        [stats[1]],
        [stats[1]],
        [["size"]],
        [stats[2], "Headshots"],
    )
    acc = weapon_fires.merge(strafe_fires, how="outer").fillna(0)
    acc = acc.merge(hits, how="outer").fillna(0)
    acc = acc.merge(headshots, how="outer").fillna(0)
    acc["Strafe%"] = acc["Strafe Fires"] / acc["Weapon Fires"]
    acc["ACC%"] = acc["Hits"] / acc["Weapon Fires"]
    acc["HS ACC%"] = acc["Headshots"] / acc["Weapon Fires"]
    acc = acc[[stats[2], "Weapon Fires", "Strafe%", "ACC%", "HS ACC%"]]
    acc.sort_values(by="ACC%", ascending=False, inplace=True)
    acc.reset_index(drop=True, inplace=True)
    return acc


accuracy(data_df["damages"], data_df["weaponFires"])

,Player,Weapon Fires,Strafe%,ACC%,HS ACC%
0,device,337,0.005935,0.267062,0.023739
1,Stewie2K,284,0.024648,0.242958,0.035211
2,Xyp9x,518,0.007722,0.200772,0.021236
3,EliGE,481,0.008316,0.185031,0.022869
4,Magisk,453,0.050773,0.174393,0.017660
5,dupreeh,445,0.035955,0.168539,0.022472
6,gla1ve,580,0.048276,0.162069,0.020690
7,NAF,491,0.006110,0.160896,0.024440
8,Twistzz,370,0.043243,0.143243,0.021622
9,nitr0,516,0.029070,0.127907,0.034884


Below is an example of the functionality of calculating statistics for each player or for each team. The default outputabove has statistics calculated for each player whereas the output below has statistics calculated for each team.

In [6]:
accuracy(data_df["damages"], data_df["weaponFires"], True)

,Team,Weapon Fires,Strafe%,ACC%,HS ACC%
0,Astralis,2333,0.031290,0.189456,0.021003
1,Team Liquid,2142,0.021008,0.166200,0.027544


# `kast()`
`kast()` takes in kill data, a string representing the combination of KAST statistics to use, a boolean specifying whether to count flash assists as assists, and filters for each group of data, and returns a DataFrame with KAST percentage and statistics, by player.

In [7]:
def kast(
    kill_data: pd.DataFrame,
    kast_string: str = "KAST",
    flash_assists: bool = True,
    kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
    death_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    columns = ["Player", f"{kast_string.upper()}%"]
    kast_counts = {}
    kast_rounds = {}
    for stat in kast_string.upper():
        columns.append(stat)
    killers = calc_stats(
        kill_data.loc[kill_data["attackerTeam"] != kill_data["victimTeam"]],
        kill_filters,
        ["roundNum"],
        ["attackerName"],
        [["sum"]],
        ["RoundNum", "Killers"],
    )
    victims = calc_stats(
        kill_data,
        kill_filters,
        ["roundNum"],
        ["victimName"],
        [["sum"]],
        ["RoundNum", "Victims"],
    )
    assisters = calc_stats(
        kill_data.loc[kill_data["assisterTeam"] !=
                      kill_data["victimTeam"]].fillna(""),
        kill_filters,
        ["roundNum"],
        ["assisterName"],
        [["sum"]],
        ["RoundNum", "Assisters"],
    )
    traded = calc_stats(
        kill_data.loc[
            (kill_data["attackerTeam"] != kill_data["victimTeam"])
            & (kill_data["isTrade"] == True)
        ].fillna(""),
        kill_filters,
        ["roundNum"],
        ["playerTradedName"],
        [["sum"]],
        ["RoundNum", "Traded"],
    )
    if flash_assists:
        flash_assisters = calc_stats(
            kill_data.loc[
                kill_data["flashThrowerTeam"] != kill_data["victimTeam"]
            ].fillna(""),
            kill_filters,
            ["roundNum"],
            ["flashThrowerName"],
            [["sum"]],
            ["RoundNum", "Flash Assisters"],
        )
        assisters = assisters.merge(flash_assisters, on="RoundNum")
        assisters["Assisters"] = assisters["Assisters"] + \
            assisters["Flash Assisters"]
        assisters = assisters[["RoundNum", "Assisters"]]
    kast_data = killers.merge(assisters, how="outer").fillna("")
    kast_data = kast_data.merge(victims, how="outer").fillna("")
    kast_data = kast_data.merge(traded, how="outer").fillna("")
    for player in kill_data["attackerName"].unique():
        kast_counts[player] = [[0, 0, 0, 0] for i in range(len(kast_data))]
        kast_rounds[player] = [0, 0, 0, 0, 0]
    for rd in kast_data.index:
        for player in kast_counts:
            if "K" in kast_string.upper():
                kast_counts[player][rd][0] = kast_data.iloc[rd]["Killers"].count(
                    player)
                kast_rounds[player][1] += kast_data.iloc[rd]["Killers"].count(
                    player)
            if "A" in kast_string.upper():
                kast_counts[player][rd][1] = kast_data.iloc[rd]["Assisters"].count(
                    player
                )
                kast_rounds[player][2] += kast_data.iloc[rd]["Assisters"].count(
                    player)
            if "S" in kast_string.upper():
                if player not in kast_data.iloc[rd]["Victims"]:
                    kast_counts[player][rd][2] = 1
                    kast_rounds[player][3] += 1
            if "T" in kast_string.upper():
                kast_counts[player][rd][3] = kast_data.iloc[rd]["Traded"].count(
                    player)
                kast_rounds[player][4] += kast_data.iloc[rd]["Traded"].count(
                    player)
    for player in kast_rounds:
        for rd in kast_counts[player]:
            if any(rd):
                kast_rounds[player][0] += 1
        kast_rounds[player][0] /= len(kast_data)
    kast = pd.DataFrame.from_dict(kast_rounds, orient="index").reset_index()
    kast.columns = ["Player", f"{kast_string.upper()}%", "K", "A", "S", "T"]
    kast = kast[columns]
    kast.fillna(0, inplace=True)
    kast.sort_values(by=f"{kast_string.upper()}%",
                     ascending=False, inplace=True)
    kast.reset_index(drop=True, inplace=True)
    return kast


kast(data_df["kills"])

,Player,KAST%,K,A,S,T
0,Xyp9x,0.678571,22,4,12,2
1,nitr0,0.678571,19,1,11,1
2,dupreeh,0.678571,17,1,12,3
3,device,0.678571,23,3,11,3
4,Stewie2K,0.642857,17,6,8,0
5,gla1ve,0.642857,17,8,12,2
6,Twistzz,0.642857,13,2,9,4
7,NAF,0.642857,17,0,9,2
8,Magisk,0.607143,16,7,9,3
9,EliGE,0.500000,18,3,7,2


# `kill_stats()`
`kill_stats()` takes in damage data, kill data, round data, weapon fire data, a boolean specifying whether to calculate statistics for each player or for each team, and filters for each group of data, and returns a DataFrame with kills, deaths, assists, flash assists, plus-minus, first kills, first kils plus-minus, trades, headshots, headshot percentage, accuracy percentage, headshot accuracy percentage, kill-death ratio, kills per round, and KAST percentage.

In [8]:
def kill_stats(
    damage_data: pd.DataFrame,
    kill_data: pd.DataFrame,
    round_data: pd.DataFrame,
    weapon_fire_data: pd.DataFrame,
    team: bool = False,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
    death_filters: Dict[str, Union[List[bool], List[str]]] = {},
    round_filters: Dict[str, Union[List[bool], List[str]]] = {},
    weapon_fire_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    stats = ["attackerName", "victimName",
             "assisterName", "flashThrowerName", "Player"]
    if team:
        stats = [
            "attackerTeam",
            "victimTeam",
            "assisterTeam",
            "flashThrowerTeam",
            "Team",
        ]
    kills = calc_stats(
        kill_data.loc[kill_data["attackerTeam"] != kill_data["victimTeam"]],
        kill_filters,
        [stats[0]],
        [stats[0]],
        [["size"]],
        [stats[4], "K"],
    )
    deaths = calc_stats(
        kill_data,
        death_filters,
        [stats[1]],
        [stats[1]],
        [["size"]],
        [stats[4], "D"],
    )
    assists = calc_stats(
        kill_data.loc[kill_data["assisterTeam"] != kill_data["victimTeam"]],
        kill_filters,
        [stats[2]],
        [stats[2]],
        [["size"]],
        [stats[4], "A"],
    )
    flash_assists = calc_stats(
        kill_data.loc[kill_data["flashThrowerTeam"]
                      != kill_data["victimTeam"]],
        kill_filters,
        [stats[3]],
        [stats[3]],
        [["size"]],
        [stats[4], "FA"],
    )
    first_kills = calc_stats(
        kill_data.loc[
            (kill_data["attackerTeam"] != kill_data["victimTeam"])
            & (kill_data["isFirstKill"] == True)
        ],
        kill_filters,
        [stats[0]],
        [stats[0]],
        [["size"]],
        [stats[4], "FK"],
    )
    first_deaths = calc_stats(
        kill_data.loc[
            (kill_data["attackerTeam"] != kill_data["victimTeam"])
            & (kill_data["isFirstKill"] == True)
        ],
        kill_filters,
        [stats[1]],
        [stats[1]],
        [["size"]],
        [stats[4], "FD"],
    )
    headshots = calc_stats(
        kill_data.loc[
            (kill_data["attackerTeam"] != kill_data["victimTeam"])
            & (kill_data["isHeadshot"] == True)
        ],
        kill_filters,
        [stats[0]],
        [stats[0]],
        [["size"]],
        [stats[4], "HS"],
    )
    headshot_pct = calc_stats(
        kill_data.loc[kill_data["attackerTeam"] != kill_data["victimTeam"]],
        kill_filters,
        [stats[0]],
        ["isHeadshot"],
        [["mean"]],
        [stats[4], "HS%"],
    )
    if not team:
        acc_stats = accuracy(
            damage_data, weapon_fire_data, False, damage_filters, weapon_fire_filters
        )
    else:
        acc_stats = accuracy(
            damage_data, weapon_fire_data, True, damage_filters, weapon_fire_filters
        )
    kast_stats = kast(kill_data, "KAST", kill_filters, death_filters)
    kill_stats = kills.merge(deaths, how="outer").fillna(0)
    kill_stats = kill_stats.merge(assists, how="outer").fillna(0)
    kill_stats = kill_stats.merge(flash_assists, how="outer").fillna(0)
    kill_stats = kill_stats.merge(first_kills, how="outer").fillna(0)
    kill_stats = kill_stats.merge(first_deaths, how="outer").fillna(0)
    kill_stats = kill_stats.merge(headshots, how="outer").fillna(0)
    kill_stats = kill_stats.merge(headshot_pct, how="outer").fillna(0)
    kill_stats = kill_stats.merge(acc_stats, how="outer").fillna(0)
    if not team:
        kill_stats = kill_stats.merge(kast_stats, how="outer").fillna(0)
    kill_stats["+/-"] = kill_stats["K"] - kill_stats["D"]
    kill_stats["KDR"] = kill_stats["K"] / kill_stats["D"]
    kill_stats["KPR"] = kill_stats["K"] / len(
        calc_stats(round_data, round_filters, [], [], [], round_data.columns)
    )
    kill_stats["FK +/-"] = kill_stats["FK"] - kill_stats["FD"]
    int_stats = ["K", "D", "A", "FA", "+/-", "FK", "FK +/-", "HS", "T"]
    if team:
        int_stats = int_stats[0:-1]
    kill_stats[int_stats] = kill_stats[int_stats].astype(int)
    kill_stats["HS%"] = kill_stats["HS%"].astype(float)
    order = [
        stats[4],
        "K",
        "D",
        "A",
        "FA",
        "+/-",
        "FK",
        "FK +/-",
        "T",
        "HS",
        "HS%",
        "ACC%",
        "HS ACC%",
        "KDR",
        "KPR",
        "KAST%",
    ]
    if team:
        order = order[0:8] + order[9:-1]
    kill_stats = kill_stats[order]
    kill_stats.sort_values(by="K", ascending=False, inplace=True)
    kill_stats.reset_index(drop=True, inplace=True)
    return kill_stats


kill_stats(data_df["damages"], data_df["kills"],
           data_df["rounds"], data_df["weaponFires"])

,Player,K,D,A,FA,+/-,FK,FK +/-,T,HS,HS%,ACC%,HS ACC%,KDR,KPR,KAST%
0,device,23,17,2,1,6,2,0,3,7,0.304348,0.267062,0.023739,1.352941,0.821429,0.678571
1,Xyp9x,22,16,3,1,6,3,2,2,7,0.318182,0.200772,0.021236,1.375000,0.785714,0.678571
2,nitr0,19,17,1,0,2,2,0,1,16,0.842105,0.127907,0.034884,1.117647,0.678571,0.678571
3,EliGE,18,21,3,0,-3,6,2,2,9,0.500000,0.185031,0.022869,0.857143,0.642857,0.500000
4,NAF,17,19,0,0,-2,4,4,2,5,0.294118,0.160896,0.024440,0.894737,0.607143,0.642857
5,Stewie2K,17,20,6,0,-3,2,-1,0,10,0.588235,0.242958,0.035211,0.850000,0.607143,0.642857
6,dupreeh,17,16,1,0,1,2,0,3,9,0.529412,0.168539,0.022472,1.062500,0.607143,0.678571
7,gla1ve,17,16,8,0,1,1,-6,2,9,0.529412,0.162069,0.020690,1.062500,0.607143,0.642857
8,Magisk,16,19,5,2,-3,3,-2,3,6,0.375000,0.174393,0.017660,0.842105,0.571429,0.607143
9,Twistzz,13,19,2,0,-6,3,1,4,6,0.461538,0.143243,0.021622,0.684211,0.464286,0.642857


# `adr()`
`adr()` takes in damage data, round data, a boolean specifying whether to calculate statistics for each player or for each team, and filters for each group of data, and returns a DataFrame with normalized and raw ADR.

In [9]:
def adr(
    damage_data: pd.DataFrame,
    round_data: pd.DataFrame,
    team: bool = False,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    round_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    stats = ["attackerName", "Player"]
    if team:
        stats = ["attackerTeam", "Team"]
    adr = calc_stats(
        damage_data.loc[damage_data["attackerTeam"]
                        != damage_data["victimTeam"]],
        damage_filters,
        [stats[0]],
        ["hpDamageTaken", "hpDamage"],
        [["sum"], ["sum"]],
        [stats[1], "Norm ADR", "Raw ADR"],
    )
    adr["Norm ADR"] = adr["Norm ADR"] / len(
        calc_stats(round_data, round_filters, [], [], [], round_data.columns)
    )
    adr["Raw ADR"] = adr["Raw ADR"] / len(
        calc_stats(round_data, round_filters, [], [], [], round_data.columns)
    )
    adr.sort_values(by="Norm ADR", ascending=False, inplace=True)
    adr.reset_index(drop=True, inplace=True)
    return adr


adr(data_df["damages"], data_df["rounds"])

,Player,Norm ADR,Raw ADR
0,Xyp9x,90.000000,110.821429
1,gla1ve,86.285714,110.392857
2,EliGE,85.500000,103.428571
3,Stewie2K,83.071429,102.714286
4,NAF,77.035714,94.464286
5,device,73.535714,95.285714
6,nitr0,63.571429,86.321429
7,dupreeh,61.785714,87.607143
8,Magisk,60.000000,74.178571
9,Twistzz,48.285714,56.928571


# `rating()`
`rating()` takes in damage data, kill data, round data, and filters for each group of data, and returns a DataFrame with estimated Impact and Rating (similar to what you see on HLTV). We use the methodology in https://flashed.gg/posts/reverse-engineering-hltv-rating/.

In [10]:
def rating(
    damage_data: pd.DataFrame,
    kill_data: pd.DataFrame,
    round_data: pd.DataFrame,
    kast_string: str = "KAST",
    flash_assists: bool = True,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    death_filters: Dict[str, Union[List[bool], List[str]]] = {},
    kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
    round_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    """Returns a dataframe with an HLTV-esque rating, found by doing:

    Rating = 0.0073*KAST + 0.3591*KPR + -0.5329*DPR + 0.2372*Impact + 0.0032*ADR + 0.1587
    where Impact = 2.13*KPR + 0.42*Assist per Round -0.41

    https://flashed.gg/posts/reverse-engineering-hltv-rating/

    Args:
        damage_data: A dataframe with damage data.
        kill_data: A dataframe with damage data.
        round_data: A dataframe with round data.
        kast_string: A string specifying which combination of KAST statistics
            to use.
        flash_assists: A boolean specifying if flash assists are to be
            counted as assists or not.
        damage_filters: A dictionary where the keys are the columns of the
            dataframe represented by damage_data to filter the damage data by
            and the values are lists that contain the column filters.
        death_filters: A dictionary where the keys are the columns of the
            dataframe represented by kill_data to filter the death data by and
            the values are lists that contain the column filters.
        kill_filters: A dictionary where the keys are the columns of the
            dataframe represented by kill_data to filter the kill data by and
            the values are lists that contain the column filters.
        round_filters: A dictionary where the keys are the columns of the
            dataframe represented by round_data to filter the round data by and
            the values are lists that contain the column filters.
    """
    stats_kills = ["attackerName", "victimName",
                   "assisterName", "flashThrowerName", "Player"]
    kast_stats = kast(kill_data, "KAST", kill_filters, death_filters)
    kast_stats = kast_stats[["Player", "KAST%"]]
    kast_stats.columns = ["Player", "KAST"]
    adr_stats = adr(damage_data, round_data, damage_filters, round_filters)
    adr_stats = adr_stats[["Player", "Norm ADR"]]
    adr_stats.columns = ["Player", "ADR"]
    stats = ["attackerName", "Player"]
    kills = calc_stats(
        kill_data.loc[kill_data["attackerTeam"] != kill_data["victimTeam"]],
        kill_filters,
        [stats_kills[0]],
        [stats_kills[0]],
        [["size"]],
        [stats_kills[4], "K"],
    )
    deaths = calc_stats(
        kill_data, death_filters, [stats_kills[1]], [
            stats_kills[1]], [["size"]], [stats_kills[4], "D"],
    )
    assists = calc_stats(
        kill_data.loc[kill_data["assisterTeam"] != kill_data["victimTeam"]],
        kill_filters,
        [stats_kills[2]],
        [stats_kills[2]],
        [["size"]],
        [stats_kills[4], "A"],
    )
    kill_stats = kills.merge(deaths, how="outer").fillna(0)
    kill_stats = kill_stats.merge(assists, how="outer").fillna(0)
    kill_stats["KPR"] = kill_stats["K"] / len(
        calc_stats(round_data, round_filters, [], [], [], round_data.columns)
    )
    kill_stats["DPR"] = kill_stats["D"] / len(
        calc_stats(round_data, round_filters, [], [], [], round_data.columns)
    )
    kill_stats["APR"] = kill_stats["A"] / len(
        calc_stats(round_data, round_filters, [], [], [], round_data.columns)
    )
    kill_stats = kill_stats[["Player", "KPR", "DPR", "APR"]]
    kill_stats = kill_stats.merge(adr_stats, how="outer").fillna(0)
    kill_stats = kill_stats.merge(kast_stats, how="outer").fillna(0)
    kill_stats["Impact"] = 2.13*kill_stats["KPR"] + \
        0.42*kill_stats["APR"] - 0.41
    kill_stats["Rating"] = 0.73*kill_stats["KAST"] + 0.3591*kill_stats["KPR"] - 0.5329 * \
        kill_stats["DPR"] + 0.2372*kill_stats["Impact"] + \
        0.0032*kill_stats["ADR"] + 0.1587
    kill_stats = kill_stats[["Player", "Impact", "Rating"]]
    kill_stats.sort_values(by="Rating", ascending=False, inplace=True)
    kill_stats.reset_index(drop=True, inplace=True)
    return kill_stats


rating(data_df["damages"], data_df["kills"], data_df["rounds"])

,Player,Impact,Rating
0,Xyp9x,1.308571,1.230086
1,device,1.369643,1.185679
2,gla1ve,1.003214,1.055573
3,nitr0,1.050357,1.026759
4,dupreeh,0.898214,0.978339
5,Stewie2K,0.973214,0.962043
6,NAF,0.883214,0.940413
7,EliGE,1.004286,0.866692
8,Magisk,0.882143,0.846748
9,Twistzz,0.608929,0.732052


# `util_dmg()`
`util_dmg()` takes in damage data, grenade data, a boolean specifying whether to calculate statistics for each player or for each team, and filters for each group of data, and returns a DataFrame with given utility damage, utility damage, grenades thrown, given utility damage per grenade, and utility damage per grenade.

In [11]:
def util_dmg(
    damage_data: pd.DataFrame,
    grenade_data: pd.DataFrame,
    team: bool = False,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    stats = ["attackerName", "throwerName", "Player"]
    if team:
        stats = ["attackerTeam", "throwerTeam", "Team"]
    util_dmg = calc_stats(
        damage_data.loc[
            (damage_data["attackerTeam"] != damage_data["victimTeam"])
            & (
                damage_data["weapon"].isin(
                    ["HE Grenade", "Incendiary Grenade", "Molotov"]
                )
            )
        ],
        damage_filters,
        [stats[0]],
        ["hpDamageTaken", "hpDamage"],
        [["sum"], ["sum"]],
        [stats[2], "Given UD", "UD"],
    )
    nades_thrown = calc_stats(
        grenade_data.loc[
            grenade_data["grenadeType"].isin(
                ["HE Grenade", "Incendiary Grenade", "Molotov"]
            )
        ],
        grenade_filters,
        [stats[1]],
        [stats[1]],
        [["size"]],
        [stats[2], "Nades Thrown"],
    )
    util_dmg_stats = util_dmg.merge(nades_thrown, how="outer").fillna(0)
    util_dmg_stats["Given UD Per Nade"] = (
        util_dmg_stats["Given UD"] / util_dmg_stats["Nades Thrown"]
    )
    util_dmg_stats["UD Per Nade"] = (
        util_dmg_stats["UD"] / util_dmg_stats["Nades Thrown"]
    )
    util_dmg_stats.sort_values(by="Given UD", ascending=False, inplace=True)
    util_dmg_stats.reset_index(drop=True, inplace=True)
    return util_dmg_stats


util_dmg(data_df["damages"], data_df["grenades"])

,Player,Given UD,UD,Nades Thrown,Given UD Per Nade,UD Per Nade
0,gla1ve,175,175,23,7.608696,7.608696
1,device,157,171,21,7.476190,8.142857
2,Xyp9x,153,153,34,4.500000,4.500000
3,Magisk,124,161,32,3.875000,5.031250
4,EliGE,119,119,26,4.576923,4.576923
5,Stewie2K,118,118,18,6.555556,6.555556
6,nitr0,83,83,18,4.611111,4.611111
7,NAF,76,76,21,3.619048,3.619048
8,dupreeh,69,70,19,3.631579,3.684211
9,Twistzz,2,2,29,0.068966,0.068966


# `flash_stats()`
`flash_stats()` takes in flash data, grenade data, kill data, a boolean specifying whether to calculate statistics for each player or for each team, and filters for each group of data, and returns a DataFrame with enemy flashes, flash assists, enemy blind time, team flashes, flashes thrown, enemy flashes per throw, and enemy blind time per enemy.

In [12]:
def flash_stats(
    flash_data: pd.DataFrame,
    grenade_data: pd.DataFrame,
    kill_data: pd.DataFrame,
    team: bool = False,
    flash_filters: Dict[str, Union[List[bool], List[str]]] = {},
    grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
    kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    stats = ["attackerName", "flashThrowerName", "throwerName", "Player"]
    if team:
        stats = ["attackerTeam", "flashThrowerTeam", "throwerTeam", "Team"]
    enemy_flashes = calc_stats(
        flash_data.loc[flash_data["attackerTeam"] != flash_data["playerTeam"]],
        flash_filters,
        [stats[0]],
        [stats[0]],
        [["size"]],
        [stats[3], "EF"],
    )
    flash_assists = calc_stats(
        kill_data.loc[kill_data["flashThrowerTeam"]
                      != kill_data["victimTeam"]],
        kill_filters,
        [stats[1]],
        [stats[1]],
        [["size"]],
        [stats[3], "FA"],
    )
    blind_time = calc_stats(
        flash_data.loc[flash_data["attackerTeam"] != flash_data["playerTeam"]],
        flash_filters,
        [stats[0]],
        ["flashDuration"],
        [["sum"]],
        [stats[3], "EBT"],
    )
    team_flashes = calc_stats(
        flash_data.loc[flash_data["attackerTeam"] == flash_data["playerTeam"]],
        flash_filters,
        [stats[0]],
        [stats[0]],
        [["size"]],
        [stats[3], "TF"],
    )
    flashes_thrown = calc_stats(
        grenade_data.loc[grenade_data["grenadeType"] == "Flashbang"],
        flash_filters,
        [stats[2]],
        [stats[2]],
        [["size"]],
        [stats[3], "Flashes Thrown"],
    )
    flash_stats = enemy_flashes.merge(flash_assists, how="outer").fillna(0)
    flash_stats = flash_stats.merge(blind_time, how="outer").fillna(0)
    flash_stats = flash_stats.merge(team_flashes, how="outer").fillna(0)
    flash_stats = flash_stats.merge(flashes_thrown, how="outer").fillna(0)
    flash_stats["EF Per Throw"] = flash_stats["EF"] / \
        flash_stats["Flashes Thrown"]
    flash_stats["EBT Per Enemy"] = flash_stats["EBT"] / flash_stats["EF"]
    flash_stats["FA"] = flash_stats["FA"].astype(int)
    flash_stats.sort_values(by="EF", ascending=False, inplace=True)
    flash_stats.reset_index(drop=True, inplace=True)
    return flash_stats


flash_stats(data_df["flashes"], data_df["grenades"], data_df["kills"])

,Player,EF,FA,EBT,TF,Flashes Thrown,EF Per Throw,EBT Per Enemy
0,gla1ve,19,0,45.626932,8,19,1.000000,2.401417
1,Xyp9x,17,1,42.747928,9,19,0.894737,2.514584
2,Magisk,16,2,44.105169,22,15,1.066667,2.756573
3,dupreeh,14,0,22.728529,19,12,1.166667,1.623466
4,device,13,1,43.219197,8,14,0.928571,3.324554
5,Twistzz,10,0,13.988292,13,11,0.909091,1.398829
6,nitr0,8,0,14.272723,6,12,0.666667,1.784090
7,NAF,7,0,8.220740,5,6,1.166667,1.174391
8,Stewie2K,7,0,15.863331,2,7,1.000000,2.266190
9,EliGE,3,0,12.819579,4,6,0.500000,4.273193


# `bomb_stats()`
`bomb_stats()` takes in bomb data and bomb data filters, and returns a DataFrame with bomb plants, defuses, and defuse percentage, by side and bombsite.

In [13]:
def bomb_stats(
    bomb_data: pd.DataFrame,
    bomb_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    team_one = bomb_data["playerTeam"].unique()[0]
    team_two = bomb_data["playerTeam"].unique()[1]
    team_one_plants = calc_stats(
        bomb_data.loc[
            (bomb_data["bombAction"] == "plant") & (
                bomb_data["playerTeam"] == team_one)
        ],
        bomb_filters,
        ["bombSite"],
        ["bombSite"],
        [["size"]],
        ["Bombsite", f"{team_one} Plants"],
    )
    team_two_plants = calc_stats(
        bomb_data.loc[
            (bomb_data["bombAction"] == "plant") & (
                bomb_data["playerTeam"] == team_two)
        ],
        bomb_filters,
        ["bombSite"],
        ["bombSite"],
        [["size"]],
        ["Bombsite", f"{team_two} Plants"],
    )
    team_one_defuses = calc_stats(
        bomb_data.loc[
            (bomb_data["bombAction"] == "defuse")
            & (bomb_data["playerTeam"] == team_one)
        ],
        bomb_filters,
        ["bombSite"],
        ["bombSite"],
        [["size"]],
        ["Bombsite", f"{team_one} Defuses"],
    )
    team_two_defuses = calc_stats(
        bomb_data.loc[
            (bomb_data["bombAction"] == "defuse")
            & (bomb_data["playerTeam"] == team_two)
        ],
        bomb_filters,
        ["bombSite"],
        ["bombSite"],
        [["size"]],
        ["Bombsite", f"{team_two} Defuses"],
    )
    bomb_stats = team_one_plants.merge(team_two_defuses, how="outer").fillna(0)
    bomb_stats[f"{team_two} Defuse %"] = (
        bomb_stats[f"{team_two} Defuses"] / bomb_stats[f"{team_one} Plants"]
    )
    bomb_stats = bomb_stats.merge(team_two_plants, how="outer").fillna(0)
    bomb_stats = bomb_stats.merge(team_one_defuses, how="outer").fillna(0)
    bomb_stats[f"{team_one} Defuse %"] = (
        bomb_stats[f"{team_one} Defuses"] / bomb_stats[f"{team_two} Plants"]
    )
    bomb_stats.loc[2] = [
        "A and B",
        bomb_stats[f"{team_one} Plants"].sum(),
        bomb_stats[f"{team_two} Defuses"].sum(),
        (
            bomb_stats[f"{team_two} Defuses"].sum()
            / bomb_stats[f"{team_one} Plants"].sum()
        ),
        bomb_stats[f"{team_two} Plants"].sum(),
        bomb_stats[f"{team_one} Defuses"].sum(),
        (
            bomb_stats[f"{team_one} Defuses"].sum()
            / bomb_stats[f"{team_two} Plants"].sum()
        ),
    ]
    bomb_stats.fillna(0, inplace=True)
    bomb_stats.iloc[:, [1, 2, 4, 5]
                    ] = bomb_stats.iloc[:, [1, 2, 4, 5]].astype(int)
    return bomb_stats


bomb_stats(data_df["bombEvents"])

,Bombsite,Astralis Plants,Team Liquid Defuses,Team Liquid Defuse %,Team Liquid Plants,Astralis Defuses,Astralis Defuse %
0,A,1,0,0.000000,5,1,0.2
1,B,7,1,0.142857,3,3,1.0
2,A and B,8,1,0.125000,8,4,0.5


# `econ_stats()`
`econ_stats()` takes in round data and round data filters, and returns a DataFrame with buy type, average equipment value, average cash, and average spend, by side.

In [14]:
def econ_stats(
    round_data: pd.DataFrame,
    round_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    ct_stats = calc_stats(
        round_data,
        round_filters,
        ["ctTeam"],
        ["ctStartEqVal", "ctRoundStartMoney", "ctSpend"],
        [["mean"], ["mean"], ["mean"]],
        ["Side", "Avg EQ Value", "Avg Cash", "Avg Spend"],
    )
    ct_stats["Side"] = ct_stats["Side"] + " CT"
    ct_buys = calc_stats(
        round_data,
        round_filters,
        ["ctTeam", "ctBuyType"],
        ["ctBuyType"],
        [["size"]],
        ["Side", "Buy Type", "Counts"],
    )
    ct_buys = ct_buys.pivot(index="Side", columns="Buy Type", values="Counts")
    ct_buys.reset_index(inplace=True)
    ct_buys.rename_axis(None, axis=1, inplace=True)
    ct_buys["Side"] = ct_buys["Side"] + " CT"
    t_stats = calc_stats(
        round_data,
        round_filters,
        ["tTeam"],
        ["tStartEqVal", "tRoundStartMoney", "tSpend"],
        [["mean"], ["mean"], ["mean"]],
        ["Side", "Avg EQ Value", "Avg Cash", "Avg Spend"],
    )
    t_stats["Side"] = t_stats["Side"] + " T"
    t_buys = calc_stats(
        round_data,
        round_filters,
        ["tTeam", "tBuyType"],
        ["tBuyType"],
        [["size"]],
        ["Side", "Buy Type", "Counts"],
    )
    t_buys = t_buys.pivot(index="Side", columns="Buy Type", values="Counts")
    t_buys.reset_index(inplace=True)
    t_buys.rename_axis(None, axis=1, inplace=True)
    t_buys["Side"] = t_buys["Side"] + " T"
    econ_buys = ct_buys.append(t_buys)
    econ_stats = ct_stats.append(t_stats)
    econ_stats = econ_buys.merge(econ_stats, how="outer")
    econ_stats.fillna(0, inplace=True)
    econ_stats.iloc[:, 1:] = econ_stats.iloc[:, 1:].astype(int)
    return econ_stats


econ_stats(data_df["rounds"])

,Side,Full Buy,Full Eco,Semi Buy,Semi Eco,Avg EQ Value,Avg Cash,Avg Spend
0,Astralis CT,11,1,1,0,26176,38473,12484
1,Team Liquid CT,12,2,0,1,23656,20206,13530
2,Astralis T,9,2,4,0,18926,20643,13296
3,Team Liquid T,9,2,0,2,17557,17934,14061


# `kill_breakdown()`
`kill_breakdown()` takes in kill data, a boolean specifying whether to calculate statistics for each player or for each team, and kill data filters, and returns a DataFrame with kills by weapon type.

In [15]:
# Helper function for kill_breakdown()
def weapon_type(weapon: str) -> str:
    if weapon in ["Knife"]:
        return "Melee Kills"
    elif weapon in [
        "CZ-75 Auto",
        "Desert Eagle",
        "Dual Berettas",
        "Five-SeveN",
        "Glock-18",
        "P2000",
        "P250",
        "R8 Revolver",
        "Tec-9",
        "USP-S",
    ]:
        return "Pistol Kills"
    elif weapon in ["MAG-7", "Nova", "Sawed-Off", "XM1014"]:
        return "Shotgun Kills"
    elif weapon in ["MAC-10", "MP5-SD", "MP7", "MP9", "P90", "PP-Bizon", "UMP-45"]:
        return "SMG Kills"
    elif weapon in ["AK-47", "AUG", "FAMAS", "Galil AR", "M4A1-S", "M4A4", "SG 553"]:
        return "Assault Rifle Kills"
    elif weapon in ["M249", "Negev"]:
        return "Machine Gun Kills"
    elif weapon in ["AWP", "G3SG1", "SCAR-20", "SSG 08"]:
        return "Sniper Rifle Kills"
    else:
        return "Utility Kills"


def kill_breakdown(
    kill_data: pd.DataFrame,
    team: bool = False,
    kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    stats = ["attackerName", "Player"]
    if team:
        stats = ["attackerTeam", "Team"]
    kill_breakdown = kill_data.loc[
        kill_data["attackerTeam"] != kill_data["victimTeam"]
    ].copy()
    kill_breakdown["Kills Type"] = kill_breakdown.apply(
        lambda row: weapon_type(row["weapon"]), axis=1
    )
    kill_breakdown = calc_stats(
        kill_breakdown,
        kill_filters,
        [stats[0], "Kills Type"],
        [stats[0]],
        [["size"]],
        [stats[1], "Kills Type", "Kills"],
    )
    kill_breakdown = kill_breakdown.pivot(
        index=stats[1], columns="Kills Type", values="Kills"
    )
    for col in [
        "Melee Kills",
        "Pistol Kills",
        "Shotgun Kills",
        "SMG Kills",
        "Assault Rifle Kills",
        "Machine Gun Kills",
        "Sniper Rifle Kills",
        "Utility Kills",
    ]:
        if not col in kill_breakdown.columns:
            kill_breakdown.insert(0, col, 0)
        kill_breakdown[col].fillna(0, inplace=True)
        kill_breakdown[col] = kill_breakdown[col].astype(int)
    kill_breakdown["Total Kills"] = kill_breakdown.iloc[0:].sum(axis=1)
    kill_breakdown.reset_index(inplace=True)
    kill_breakdown.rename_axis(None, axis=1, inplace=True)
    kill_breakdown = kill_breakdown[
        [
            stats[1],
            "Melee Kills",
            "Pistol Kills",
            "Shotgun Kills",
            "SMG Kills",
            "Assault Rifle Kills",
            "Machine Gun Kills",
            "Sniper Rifle Kills",
            "Utility Kills",
            "Total Kills",
        ]
    ]
    kill_breakdown.sort_values(by="Total Kills", ascending=False, inplace=True)
    kill_breakdown.reset_index(drop=True, inplace=True)
    return kill_breakdown


kill_breakdown(data_df["kills"])

,Player,Melee Kills,Pistol Kills,Shotgun Kills,SMG Kills,Assault Rifle Kills,Machine Gun Kills,Sniper Rifle Kills,Utility Kills,Total Kills
0,device,1,1,0,0,17,0,2,2,23
1,Xyp9x,0,3,0,0,18,0,0,1,22
2,nitr0,0,2,0,2,14,0,0,1,19
3,EliGE,0,2,0,2,14,0,0,0,18
4,NAF,0,1,0,1,6,0,7,2,17
5,Stewie2K,0,5,0,0,12,0,0,0,17
6,dupreeh,0,1,0,1,14,0,0,1,17
7,gla1ve,0,5,0,0,11,0,0,1,17
8,Magisk,0,0,0,3,12,0,0,1,16
9,Twistzz,0,1,0,1,9,0,0,2,13


# `util_dmg_breakdown()`
`util_dmg_breakdown()` takes in damage data, grenade data, a boolean specifying whether to calculate statistics for each player or for each team, and filters for each group of data, and returns a DataFrame with given utility damage, utility damage, grenades thrown, given utility damage per grenade, and utility damage per grenade, by grenade type.

In [16]:
def util_dmg_breakdown(
    damage_data: pd.DataFrame,
    grenade_data: pd.DataFrame,
    team: bool = False,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    stats = ["attackerName", "throwerName", "Player"]
    if team:
        stats = ["attackerTeam", "throwerTeam", "Team"]
    util_dmg = calc_stats(
        damage_data.loc[
            (damage_data["attackerTeam"] != damage_data["victimTeam"])
            & (
                damage_data["weapon"].isin(
                    ["HE Grenade", "Incendiary Grenade", "Molotov"]
                )
            )
        ],
        damage_filters,
        [stats[0], "weapon"],
        ["hpDamageTaken", "hpDamage"],
        [["sum"], ["sum"]],
        [stats[2], "Nade Type", "Given UD", "UD"],
    )
    nades_thrown = calc_stats(
        grenade_data.loc[
            grenade_data["grenadeType"].isin(
                ["HE Grenade", "Incendiary Grenade", "Molotov"]
            )
        ],
        grenade_filters,
        [stats[1], "grenadeType"],
        [stats[1]],
        [["size"]],
        [stats[2], "Nade Type", "Nades Thrown"],
    )
    util_dmg_breakdown = util_dmg.merge(
        nades_thrown, how="outer", on=[stats[2], "Nade Type"]
    ).fillna(0)
    util_dmg_breakdown["Given UD Per Nade"] = (
        util_dmg_breakdown["Given UD"] / util_dmg_breakdown["Nades Thrown"]
    )
    util_dmg_breakdown["UD Per Nade"] = (
        util_dmg_breakdown["UD"] / util_dmg_breakdown["Nades Thrown"]
    )
    util_dmg_breakdown.sort_values(
        by=[stats[2], "Given UD"], ascending=[True, False], inplace=True
    )
    util_dmg_breakdown.reset_index(drop=True, inplace=True)
    return util_dmg_breakdown


util_dmg_breakdown(data_df["damages"], data_df["grenades"])

,Player,Nade Type,Given UD,UD,Nades Thrown,Given UD Per Nade,UD Per Nade
0,EliGE,HE Grenade,119.0,119.0,13,9.153846,9.153846
1,EliGE,Incendiary Grenade,0.0,0.0,6,0.000000,0.000000
2,EliGE,Molotov,0.0,0.0,7,0.000000,0.000000
3,Magisk,Molotov,76.0,76.0,7,10.857143,10.857143
4,Magisk,HE Grenade,48.0,85.0,16,3.000000,5.312500
5,Magisk,Incendiary Grenade,0.0,0.0,9,0.000000,0.000000
6,NAF,Molotov,76.0,76.0,9,8.444444,8.444444
7,NAF,HE Grenade,0.0,0.0,11,0.000000,0.000000
8,NAF,Incendiary Grenade,0.0,0.0,1,0.000000,0.000000
9,Stewie2K,HE Grenade,86.0,86.0,6,14.333333,14.333333


# `win_breakdown()`
`win_breakdown()` takes in round data and round data filters, and returns a DataFrame with win type by team.

In [17]:
def win_breakdown(
    round_data: pd.DataFrame,
    round_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    round_data_copy = round_data.copy()
    round_data_copy.replace(
        "BombDefused", "CT Bomb Defusal Wins", inplace=True)
    round_data_copy.replace("CTWin", "CT T Elim Wins", inplace=True)
    round_data_copy.replace(
        "TargetBombed", "T Bomb Detonation Wins", inplace=True)
    round_data_copy.replace(
        "TargetSaved", "CT Time Expired Wins", inplace=True)
    round_data_copy.replace("TerroristsWin", "T CT Elim Wins", inplace=True)
    win_breakdown = calc_stats(
        round_data_copy,
        round_filters,
        ["winningTeam", "roundEndReason"],
        ["roundEndReason"],
        [["size"]],
        ["Team", "RoundEndReason", "Count"],
    )
    win_breakdown = win_breakdown.pivot(
        index="Team", columns="RoundEndReason", values="Count"
    ).fillna(0)
    win_breakdown.reset_index(inplace=True)
    win_breakdown.rename_axis(None, axis=1, inplace=True)
    win_breakdown["Total CT Wins"] = win_breakdown.iloc[0:][
        list(
            set.intersection(
                set(win_breakdown.columns),
                set(["CT Bomb Defusal Wins", "CT T Elim Wins", "CT Time Expired Wins"]),
            )
        )
    ].sum(axis=1)
    win_breakdown["Total T Wins"] = win_breakdown.iloc[0:][
        list(
            set.intersection(
                set(win_breakdown.columns),
                set(["T Bomb Detonation Wins", "T CT Elim Wins"]),
            )
        )
    ].sum(axis=1)
    win_breakdown["Total Wins"] = win_breakdown.iloc[0:, 0:-2].sum(axis=1)
    win_breakdown.iloc[:, 1:] = win_breakdown.iloc[:, 1:].astype(int)
    return win_breakdown


win_breakdown(data_df["rounds"])

C:\Users\AAGRAW~1\AppData\Local\Temp/ipykernel_3060/827307204.py:43: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  win_breakdown["Total Wins"] = win_breakdown.iloc[0:, 0:-2].sum(axis=1)


,Team,CT Bomb Defusal Wins,CT T Elim Wins,CT Time Expired Wins,T Bomb Detonation Wins,T CT Elim Wins,Total CT Wins,Total T Wins,Total Wins
0,Astralis,4,5,0,2,5,9,7,16
1,Team Liquid,1,6,1,3,1,8,4,12


# `player_box_score()`
`player_box_score()` takes in damage data, flash data, grenade data, kill data, round data, weapon fire data, and filters for each group of data, and returns a player box score DataFrame containing statistics from each group of data by player.

In [18]:
def player_box_score(
    damage_data: pd.DataFrame,
    flash_data: pd.DataFrame,
    grenade_data: pd.DataFrame,
    kill_data: pd.DataFrame,
    round_data: pd.DataFrame,
    weapon_fire_data: pd.DataFrame,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    flash_filters: Dict[str, Union[List[bool], List[str]]] = {},
    grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
    kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
    death_filters: Dict[str, Union[List[bool], List[str]]] = {},
    round_filters: Dict[str, Union[List[bool], List[str]]] = {},
    weapon_fire_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    """Returns a player box score dataframe.

    Args:
       damage_data: A dataframe with damage data.
       flash_data: A dataframe with flash data.
       grenade_data: A dataframe with grenade data.
       kill_data: A dataframe with kill data.
       round_data: A dataframe with round data.
       weapon_fire_data: A dataframe with weapon fire data.
       damage_filters: A dictionary where the keys are the columns of the
           dataframe represented by damage_data to filter the damage data by
           and the values are lists that contain the column filters.
       flash_filters: A dictionary where the keys are the columns of the
           dataframe represented by flash_data to filter the flash data by
           and the values are lists that contain the column filters.
       grenade_filters: A dictionary where the keys are the columns of the
           dataframe represented by grenade_data to filter the grenade data by
           and the values are lists that contain the column filters.
       kill_filters: A dictionary where the keys are the columns of the
           dataframe represented by kill_data to filter the kill data by and
           the values are lists that contain the column filters.
       death_filters: A dictionary where the keys are the columns of the
           dataframe represented by kill_data to filter the death data by and
           the values are lists that contain the column filters.
       round_filters: A dictionary where the keys are the columns of the
           dataframe represented by round_data to filter the round data by and
           the values are lists that contain the column filters.
       weapon_fire_filters: A dictionary where the keys are the columns of the
           dataframe to filter the weapon fire data by and the values are lists
           that contain the column filters.
    """
    k_stats = kill_stats(
        damage_data,
        kill_data,
        round_data,
        weapon_fire_data,
        damage_filters,
        kill_filters,
        death_filters,
        round_filters,
        weapon_fire_filters,
    )
    k_stats = k_stats[
        ["Player", "K", "D", "A", "FA", "HS%", "ACC%", "HS ACC%", "KDR", "KAST%"]
    ]
    adr_stats = adr(damage_data, round_data, damage_filters, round_filters)
    adr_stats = adr_stats[["Player", "Norm ADR"]]
    adr_stats.columns = ["Player", "ADR"]
    ud_stats = util_dmg(damage_data, grenade_data,
                        damage_filters, grenade_filters)
    ud_stats = ud_stats[["Player", "UD", "UD Per Nade"]]
    f_stats = flash_stats(
        flash_data,
        grenade_data,
        kill_data,
        flash_filters,
        grenade_filters,
        kill_filters,
    )
    f_stats = f_stats[["Player", "EF", "EF Per Throw"]]
    rating_stats = rating(damage_data, kill_data, round_data,
                          damage_filters, death_filters, kill_filters, round_filters)
    box_score = k_stats.merge(adr_stats, how="outer").fillna(0)
    box_score = box_score.merge(ud_stats, how="outer").fillna(0)
    box_score = box_score.merge(f_stats, how="outer").fillna(0)
    box_score = box_score.merge(rating_stats, how="outer").fillna(0)
    return box_score


player_box_score(data_df["damages"], data_df["flashes"], data_df["grenades"],
                 data_df["kills"], data_df["rounds"], data_df["weaponFires"])

,Player,K,D,A,FA,HS%,ACC%,HS ACC%,KDR,KAST%,ADR,UD,UD Per Nade,EF,EF Per Throw,Impact,Rating
0,device,23,17,2,1,0.304348,0.267062,0.023739,1.352941,0.678571,73.535714,171,8.142857,13,0.928571,1.369643,1.185679
1,Xyp9x,22,16,3,1,0.318182,0.200772,0.021236,1.375000,0.678571,90.000000,153,4.500000,17,0.894737,1.308571,1.230086
2,nitr0,19,17,1,0,0.842105,0.127907,0.034884,1.117647,0.678571,63.571429,83,4.611111,8,0.666667,1.050357,1.026759
3,EliGE,18,21,3,0,0.500000,0.185031,0.022869,0.857143,0.500000,85.500000,119,4.576923,3,0.500000,1.004286,0.866692
4,NAF,17,19,0,0,0.294118,0.160896,0.024440,0.894737,0.642857,77.035714,76,3.619048,7,1.166667,0.883214,0.940413
5,Stewie2K,17,20,6,0,0.588235,0.242958,0.035211,0.850000,0.642857,83.071429,118,6.555556,7,1.000000,0.973214,0.962043
6,dupreeh,17,16,1,0,0.529412,0.168539,0.022472,1.062500,0.678571,61.785714,70,3.684211,14,1.166667,0.898214,0.978339
7,gla1ve,17,16,8,0,0.529412,0.162069,0.020690,1.062500,0.642857,86.285714,175,7.608696,19,1.000000,1.003214,1.055573
8,Magisk,16,19,5,2,0.375000,0.174393,0.017660,0.842105,0.607143,60.000000,161,5.031250,16,1.066667,0.882143,0.846748
9,Twistzz,13,19,2,0,0.461538,0.143243,0.021622,0.684211,0.642857,48.285714,2,0.068966,10,0.909091,0.608929,0.732052


# `team_box_score()`
`team_box_score()` takes in damage data, flash data, grenade data, kill data, round data, weapon fire data, and filters for each group of data, and returns a team box score DataFrame containing statistics from each group of data by team.

In [19]:
def team_box_score(
    damage_data: pd.DataFrame,
    flash_data: pd.DataFrame,
    grenade_data: pd.DataFrame,
    kill_data: pd.DataFrame,
    round_data: pd.DataFrame,
    weapon_fire_data: pd.DataFrame,
    damage_filters: Dict[str, Union[List[bool], List[str]]] = {},
    flash_filters: Dict[str, Union[List[bool], List[str]]] = {},
    grenade_filters: Dict[str, Union[List[bool], List[str]]] = {},
    kill_filters: Dict[str, Union[List[bool], List[str]]] = {},
    death_filters: Dict[str, Union[List[bool], List[str]]] = {},
    round_filters: Dict[str, Union[List[bool], List[str]]] = {},
    weapon_fire_filters: Dict[str, Union[List[bool], List[str]]] = {},
) -> pd.DataFrame:
    k_stats = kill_stats(
        damage_data,
        kill_data,
        round_data,
        weapon_fire_data,
        True,
        damage_filters,
        kill_filters,
        death_filters,
        round_filters,
        weapon_fire_filters,
    )
    acc_stats = accuracy(
        damage_data, weapon_fire_data, True, damage_filters, weapon_fire_filters
    )
    adr_stats = adr(damage_data, round_data, True,
                    damage_filters, round_filters)
    ud_stats = util_dmg(
        damage_data, grenade_data, True, damage_filters, grenade_filters
    )
    f_stats = flash_stats(
        flash_data,
        grenade_data,
        kill_data,
        True,
        flash_filters,
        grenade_filters,
        kill_filters,
    )
    e_stats = econ_stats(round_data, round_filters)
    for index in e_stats.index:
        e_stats.iloc[index, 0] = e_stats["Side"].str.rsplit(n=1)[index][0]
        rounds = e_stats.iloc[index, 1:-4].sum()
        e_stats.iloc[index, -3:] = e_stats.iloc[index, -3:] * rounds
    e_stats = e_stats.groupby(["Side"]).sum()
    e_stats.reset_index(inplace=True)
    e_stats.iloc[:, -3:] = (
        e_stats.iloc[:, -3:] / len(filter_df(round_data, round_filters))
    ).astype(int)
    e_stats.rename(columns={"Side": "Team"}, inplace=True)
    box_score = k_stats.merge(acc_stats, how="outer")
    box_score = box_score.merge(adr_stats, how="outer")
    box_score = box_score.merge(ud_stats, how="outer")
    box_score = box_score.merge(f_stats, how="outer")
    box_score = box_score.merge(e_stats, how="outer")
    box_score = box_score.merge(
        win_breakdown(round_data, round_filters), how="outer"
    ).fillna(0)
    box_score.rename(
        columns={
            "Norm ADR": "ADR",
            "Total CT Wins": "CT Wins",
            "Total T Wins": "T Wins",
            "Total Wins": "Score",
        },
        inplace=True,
    )
    score = box_score["Score"]
    ct_wins = box_score["CT Wins"]
    t_wins = box_score["T Wins"]
    box_score.drop(["Score", "CT Wins", "T Wins"], axis=1, inplace=True)
    box_score.insert(1, "Score", score)
    box_score.insert(2, "CT Wins", ct_wins)
    box_score.insert(3, "T Wins", t_wins)
    box_score = box_score.transpose()
    box_score.columns = box_score.iloc[0]
    box_score.drop("Team", inplace=True)
    box_score.rename_axis(None, axis=1, inplace=True)
    box_score = box_score.loc[
        [
            "Score",
            "CT Wins",
            "T Wins",
            "K",
            "D",
            "A",
            "FA",
            "+/-",
            "FK",
            "HS",
            "HS%",
            "Strafe%",
            "ACC%",
            "HS ACC%",
            "ADR",
            "UD",
            "Nades Thrown",
            "UD Per Nade",
            "EF",
            "Flashes Thrown",
            "EF Per Throw",
            "EBT Per Enemy",
        ],
        :,
    ].append(box_score.iloc[31:, :])
    return box_score


team_box_score(data_df["damages"], data_df["flashes"], data_df["grenades"],
               data_df["kills"], data_df["rounds"], data_df["weaponFires"])

C:\Users\AAGRAW~1\AppData\Local\Temp/ipykernel_3060/827307204.py:43: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  win_breakdown["Total Wins"] = win_breakdown.iloc[0:, 0:-2].sum(axis=1)


,Astralis,Team Liquid
Score,16,12
CT Wins,9,8
T Wins,7,4
K,95,84
D,84,96
A,19,12
FA,4,0
+/-,11,-12
FK,11,17
HS,38,46
